In [30]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

##from neural_nets import convnet_cifar10 as nnet
from neural_nets import salvar_nn

# as duas linhas abaixo são para evitar um erro de certificado causado por
# um bug na versão atual do torchvision
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def carregar_nn(model, path):
    model.load_state_dict(torch.load(path + 'model_weights.pth'))
    return model

# rede neural convolutiva    
class nnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 10)
        self.dropout = nn.Dropout (0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.pool(F.relu(self.conv6(F.relu(self.conv5(x)))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x  

In [31]:

#%% Path

n_epochs = 10

PATH = './exemplo_1a/'

#%% CPU ou GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
#%% Carregando e normalizando CIFAR10

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



cpu
Files already downloaded and verified
Files already downloaded and verified


In [32]:

if __name__ == '__main__':

    # cria a rede neural    
    net = nnet()
    # move a rede para a GPU, se houver
    net.to(device)     
    # define a função perda
    criterion = nn.CrossEntropyLoss()
    # define o otimizador a ser usado no treinamento
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    # treinamento
    for epoch in range(n_epochs):  # loop over the dataset multiple times
    
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            # inputs, labels = data
            # move para a GPU, se houver
            inputs, labels = data[0].to(device), data[1].to(device)
    
            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 100 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                    for data in testloader:
                        images, labels = data
                        # calculate outputs by running images through the network
                        outputs = net(images)
                        # the class with the highest energy is what we choose as prediction
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    
        
         
    
print('Finished Training')    
    # salvando o modelo treinado
    #salvar_nn(net, PATH)
    
    

[1,   100] loss: 2.271
Accuracy of the network on the 10000 test images: 19 %
[1,   200] loss: 1.999
Accuracy of the network on the 10000 test images: 29 %
[1,   300] loss: 1.809
Accuracy of the network on the 10000 test images: 33 %
[1,   400] loss: 1.677
Accuracy of the network on the 10000 test images: 36 %
[1,   500] loss: 1.585
Accuracy of the network on the 10000 test images: 40 %
[1,   600] loss: 1.479
Accuracy of the network on the 10000 test images: 47 %
[1,   700] loss: 1.392
Accuracy of the network on the 10000 test images: 49 %
[2,   100] loss: 1.245
Accuracy of the network on the 10000 test images: 54 %
[2,   200] loss: 1.175
Accuracy of the network on the 10000 test images: 55 %
[2,   300] loss: 1.141
Accuracy of the network on the 10000 test images: 60 %
[2,   400] loss: 1.139
Accuracy of the network on the 10000 test images: 59 %
[2,   500] loss: 1.072
Accuracy of the network on the 10000 test images: 61 %
[2,   600] loss: 1.045
Accuracy of the network on the 10000 test

In [33]:
correct = 0
total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 76 %


In [36]:

# rede neural convolutiva    
class nnet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 10)
        self.dropout = nn.Dropout (0.4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.pool(F.relu(self.conv6(F.relu(self.conv5(x)))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x  

if __name__ == '__main__':
    n_epochs = 50
    # cria a rede neural    
    net = nnet2()
    # move a rede para a GPU, se houver
    net.to(device)     
    # define a função perda
    criterion = nn.CrossEntropyLoss()
    # define o otimizador a ser usado no treinamento
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    # treinamento
    for epoch in range(n_epochs):  # loop over the dataset multiple times
    
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            # inputs, labels = data
            # move para a GPU, se houver
            inputs, labels = data[0].to(device), data[1].to(device)
    
            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 100 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                    for data in testloader:
                        images, labels = data
                        # calculate outputs by running images through the network
                        outputs = net(images)
                        # the class with the highest energy is what we choose as prediction
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    
        
         
    
print('Finished Training')    
    # salvando o modelo treinado
    #salvar_nn(net, PATH)

[1,   100] loss: 2.204
Accuracy of the network on the 10000 test images: 19 %
[1,   200] loss: 2.001
Accuracy of the network on the 10000 test images: 23 %
[1,   300] loss: 1.850
Accuracy of the network on the 10000 test images: 31 %
[1,   400] loss: 1.752
Accuracy of the network on the 10000 test images: 33 %
[1,   500] loss: 1.678
Accuracy of the network on the 10000 test images: 36 %
[1,   600] loss: 1.638
Accuracy of the network on the 10000 test images: 38 %
[1,   700] loss: 1.542
Accuracy of the network on the 10000 test images: 44 %
[2,   100] loss: 1.429
Accuracy of the network on the 10000 test images: 48 %
[2,   200] loss: 1.383
Accuracy of the network on the 10000 test images: 52 %
[2,   300] loss: 1.308
Accuracy of the network on the 10000 test images: 53 %
[2,   400] loss: 1.281
Accuracy of the network on the 10000 test images: 53 %
[2,   500] loss: 1.258
Accuracy of the network on the 10000 test images: 55 %
[2,   600] loss: 1.201
Accuracy of the network on the 10000 test

KeyboardInterrupt: 